In [4]:
import pandas as pd
from nba_api.stats.endpoints import leaguegamefinder, playbyplay, boxscoretraditionalv2
from datetime import datetime
import time

# Konfiguration för vikter
weight_config = {
    'period_weights': {1: 0.33, 2: 0.33, 3: 0.34, 4: 0}, 
    'extra_period_weight': 0.05,  # Viktning för extra perioder (övertid)
    'lead_change_weight': 0.05,  # Viktning för ledningsbyten
    'buzzer_beater_weight': 0.0,  # Viktning för buzzer-beater
    'fg3_pct_weight': 0.05,  # Viktning för FG3_PCT
    'star_performance_weight': 0.1,  # Viktning för stjärnprestationer
    'margin_weight': 0.25,  # Viktning för slutresultat mellanskillnad
    'max_total_score': 0.50  # Maximal poäng för perioder tillsammans (exkl. extra period, lead changes, buzzer-beater, FG3_PCT, star performance, och margin)
}

# Justera periodvikter så att de summerar till den önskade max total poängen
adjusted_period_weights = {k: v * weight_config['max_total_score'] for k, v in weight_config['period_weights'].items()}

def calculate_period_score(period_df, column_name, weight):
    # Ersätt 'TIE' med noll
    period_df.loc[period_df[column_name] == 'TIE', column_name] = 0
    
    # Fyll i None-värden med noll innan vi tar absolutvärdet
    period_df[column_name] = period_df[column_name].fillna(0).astype(float)
    
    # Beräkna absolutvärdet av kolumnen
    period_df['PERIODSCORE'] = period_df[column_name].abs()
    
    # Filtrera bort rader där 'SCORE' är None
    filtered_df = period_df.dropna(subset=['SCORE'])
    
    # Beräkna det totala genomsnittliga värdet på 'PERIODSCORE'
    average_periodscore = filtered_df['PERIODSCORE'].mean()
    
    # Beräkna poängen baserat på genomsnittlig poängskillnad
    if average_periodscore <= 7:
        period_score = weight * 100
    elif average_periodscore > 20:
        period_score = 0
    else:
        # Linjär minskning från 5 till 20
        period_score = weight * 100 * (20 - average_periodscore) / 13
    
    return average_periodscore, period_score

def calculate_lead_changes_score(df, weight):
    lead_changes = 0
    previous_margin = None
    
    for margin in df['SCOREMARGIN']:
        if margin == 'TIE' or pd.isna(margin):
            continue
        
        try:
            margin = float(margin)
        except ValueError:
            continue
        
        if previous_margin is not None and ((previous_margin < 0 and margin > 0) or (previous_margin > 0 and margin < 0)):
            lead_changes += 1
        previous_margin = margin
    
    # Beräkna poäng baserat på antalet ledningsbyten
    if lead_changes >= 12:
        lead_changes_score = weight * 100
    elif lead_changes <= 5:
        lead_changes_score = 0
    else:
        # Linjär minskning från 12 till 5
        lead_changes_score = weight * 100 * (lead_changes - 5) / 7
    
    return lead_changes, lead_changes_score

def calculate_buzzer_beater_score(df, weight):
    last_margin = df['SCOREMARGIN'].iloc[-1]
    
    if last_margin == 'TIE' or pd.isna(last_margin):
        return 0, 0
    
    try:
        last_margin = float(last_margin)
    except ValueError:
        return 0, 0
    
    # Samla de fem senaste värdena som inte är 'TIE' eller NaN
    recent_values = []
    for margin in df['SCOREMARGIN'].iloc[::-1]:
        if margin == 'TIE' or pd.isna(margin):
            continue
        try:
            margin = float(margin)
            recent_values.append(margin)
        except ValueError:
            continue
        if len(recent_values) == 5:
            break
    
    # Om vi har mindre än två värden, kan vi inte bedöma ett ledningsskifte
    if len(recent_values) < 2:
        return 0, 0
    
    # Kontrollera om det finns ett ledningsskifte i de senaste fem värdena
    for margin in recent_values:
        if (last_margin < 0 and margin > 0) or (last_margin > 0 and margin < 0):
            return 1, weight * 100  # En buzzer-beater upptäckt
    
    return 0, 0

def get_fg_fg3_pct_score(recent_games, game_id, weight):
    fg_pct_scores = []
    fg3_pct_scores = []

    for _, row in recent_games[recent_games['GAME_ID']==game_id].iterrows():
        fg_pct = row['FG_PCT']
        fg3_pct = row['FG3_PCT']
        
        # Beräkna FG_PCT poäng
        if fg_pct >= 0.5:
            fg_pct_score = weight * 100
        elif fg_pct <= 0.4:
            fg_pct_score = 0
        else:
            fg_pct_score = weight * 100 * (fg_pct - 0.4) / 0.1
        
        # Beräkna FG3_PCT poäng
        if fg3_pct >= 0.35:
            fg3_pct_score = weight * 100
        elif fg3_pct <= 0.25:
            fg3_pct_score = 0
        else:
            fg3_pct_score = weight * 100 * (fg3_pct - 0.25) / 0.1
        
        fg_pct_scores.append(fg_pct_score)
        fg3_pct_scores.append(fg3_pct_score)
    
    # Kombinera poängen
    combined_score = max(max(fg_pct_scores), max(fg3_pct_scores))
    return max(fg_pct_scores), max(fg3_pct_scores), combined_score

def calculate_margin_and_star_performance_score(game_id, weight_margin, weight_star):
    boxscore = boxscoretraditionalv2.BoxScoreTraditionalV2(game_id=game_id)
    boxscore_df = boxscore.get_data_frames()[0]
    
    # Beräkna margin score
    team_scores = boxscore_df.groupby('TEAM_ID')['PTS'].sum()
    score_diff = abs(team_scores.iloc[0] - team_scores.iloc[1])
    
    if score_diff >= 15:
        margin_score = 0
    elif score_diff <= 5:
        margin_score = weight_margin * 100
    else:
        margin_score = weight_margin * 100 * (15 - score_diff) / 10
    
    # Beräkna star performance score
    max_points = boxscore_df['PTS'].max()
    
    if max_points >= 35:
        star_performance_score = weight_star * 100
    elif max_points <= 20:
        star_performance_score = 0
    else:
        star_performance_score = weight_star * 100 * (max_points - 20) / 15
    
    return score_diff, margin_score, max_points, star_performance_score

def get_grade(total_score):
    if total_score >= 93:
        return 'A+'
    elif total_score >= 85:
        return 'A'
    elif total_score >= 80:
        return 'B+'
    elif total_score >= 75:
        return 'B'
    elif total_score >= 70:
        return 'C+'
    elif total_score >= 65:
        return 'C'
    else:
        return 'D'

# Hämta alla matcher och sortera dem efter datum för att plocka ut de senaste 5
def get_recent_games():
    gamefinder = leaguegamefinder.LeagueGameFinder(league_id_nullable='00')
    games = gamefinder.get_data_frames()[0]
    games = games.sort_values(by=['GAME_DATE', 'GAME_ID'], ascending=[False, False])
    games['GAME_DATE'] = pd.to_datetime(games['GAME_DATE'])
    recent_games = games.head(10)  # Hämta de senaste 5 matcherna för båda lagen
    return recent_games

# Hämta de senaste 5 spelade matcherna
recent_games = get_recent_games()

# Filtrera på rader som innehåller '@' i 'MATCHUP'
filtered_recent_games = recent_games[recent_games['MATCHUP'].str.contains('@')]

# Ta bort dubbletter baserat på 'GAME_ID'
unique_recent_games = filtered_recent_games.drop_duplicates(subset=['GAME_ID'])

results = []

# Loop genom de senaste matcherna och beräkna total scores
for _, game in unique_recent_games.iterrows():
    game_id = game['GAME_ID']
    matchup = game['MATCHUP']
    game_date = game['GAME_DATE'].strftime('%Y-%m-%d')

    pbp = playbyplay.PlayByPlay(game_id=game_id)
    pbp_df = pbp.get_data_frames()[0]
    
    # Välj specifika kolumner
    pbp_df = pbp_df[['PERIOD', 'EVENTMSGTYPE', 'EVENTMSGACTIONTYPE', 'EVENTNUM', 'SCORE', 'SCOREMARGIN', 'PCTIMESTRING']]
    pd.set_option('display.max_rows', None)
    pd.set_option('display.max_columns', None)
    print(pbp_df)
    # Konvertera SCOREMARGIN till numerisk typ, hantera fel under konvertering
    pbp_df['SCOREMARGIN'] = pd.to_numeric(pbp_df['SCOREMARGIN'], errors='coerce')
    
    total_scores = {column: 0 for column in ['Period Scores', 'Extra Periods', 'Lead Changes', 'Buzzer Beater', 'FG3_PCT', 'Star Performance', 'Margin']}
    period_scores = {period: 0 for period in adjusted_period_weights}

    for period, weight in adjusted_period_weights.items():
        # Filtrera data för den aktuella perioden
        period_df = pbp_df[pbp_df['PERIOD'] == period].copy()
        
        # Beräkna poängen för den aktuella kolumnen och perioden
        if not period_df.empty:
            average_periodscore, period_score = calculate_period_score(period_df, 'SCOREMARGIN', weight)
            period_scores[period] = period_score
    
    # Summera poängen för perioderna
    period_scores_total = sum(period_scores.values())
    total_scores['Period Scores'] = period_scores_total
    
    # Hantera extra perioder (övertid)
    extra_periods = pbp_df[pbp_df['PERIOD'] > 4]
    if not extra_periods.empty:
        extra_score = weight_config['extra_period_weight'] * 100
        total_scores['Extra Periods'] = extra_score

    # Beräkna ledningsbyten och deras poäng
    lead_changes, lead_changes_score = calculate_lead_changes_score(pbp_df, weight_config['lead_change_weight'])
    total_scores['Lead Changes'] = lead_changes_score
    
    # Beräkna buzzer-beater och deras poäng
    buzzer_beater, buzzer_beater_score = calculate_buzzer_beater_score(pbp_df, weight_config['buzzer_beater_weight'])
    total_scores['Buzzer Beater'] = buzzer_beater_score
    
    # Hämta FG_PCT och FG3_PCT och beräkna poäng baserat på båda
    max_fg_pct, max_fg3_pct, fg_combined_score = get_fg_fg3_pct_score(recent_games, game_id, weight_config['fg3_pct_weight'])
    total_scores['FG3_PCT'] = fg_combined_score
    
    # Beräkna margin och stjärnprestationer i en funktion
    score_diff, margin_score, max_points, star_performance_score = calculate_margin_and_star_performance_score(game_id, weight_config['margin_weight'], weight_config['star_performance_weight'])
    total_scores['Margin'] = margin_score
    total_scores['Star Performance'] = star_performance_score

    # Summera totalpoängen
    total_score = round(sum(total_scores.values()),1)
    grade = get_grade(total_score)

    # Lägg till resultatet för denna match
    results.append({
        'Game Date': game_date,
        'Teams': matchup,
        'Total Score': total_score,
        'Grade': grade,
        'Period Scores': period_scores_total,
        'Extra Periods': total_scores['Extra Periods'],
        'Lead Changes': total_scores['Lead Changes'],
        'Buzzer Beater': total_scores['Buzzer Beater'],
        'FG3_PCT': total_scores['FG3_PCT'],
        'Star Performance': total_scores['Star Performance'],
        'Margin': total_scores['Margin']
    })

# Skapa en DataFrame med resultaten
results_df = pd.DataFrame(results)

print(results_df)


     PERIOD  EVENTMSGTYPE  EVENTMSGACTIONTYPE  EVENTNUM      SCORE   
0         1            12                   0         2       None  \
1         1            10                   0         4       None   
2         1             5                  39         7       None   
3         1             2                   9         8       None   
4         1             4                   0         9       None   
5         1             5                  40        11       None   
6         1             1                  98        13      2 - 0   
7         1             1                   1        15      2 - 3   
8         1             1                   1        17      5 - 3   
9         1             1                   1        19      5 - 6   
10        1             5                   1        21       None   
11        1             6                   1        23       None   
12        1             1                  80        25      5 - 9   
13        1         